In [ ]:
# baseQuery là gì?

- Hãy tưởng tượng:
    - RTK Query là một ông QUẢN LÝ KHO HÀNG (Quản lý data, cache, loading...). Ông ấy chỉ biết ra lệnh " Đi lấy hàng về cho tôi"
    - Ông ấy cần một Shipper (Người vận chuyển) để thực sự chạy ra ngoài lấy hàng
    - Shipper đó chính là [baseQuery]
- Mặc định:
    - RTK Query cung cấp sẵn một Shipper tên là [fetchBaseQuery] (dùng hàm [fetch] có sẵn của trình duyệt)
    - Tuy nhiên, trong dự án thực tế, chúng ta thường dùng [axios] vì nó xin hơn (có interceptor tự động gán token, xử lý lỗi chung...)

- TÓM LẠI:
> Customizing Queries chính là việc nói với ông QUẢN LÝ KHO: "Đừng dùng Shipper mặc định nữa, hãy dùng đội xe tải Axios của tôi"

In [ ]:
# Cấu trúc chuẩn một baseQuery

- Dù bạn dùng [fetch], [axios] thì một hàm [baseQuery] bắt buộc phải tuân thủ quy tắc giao tiếp với ông QUẢN LÝ KHO như sau:

## A. Đầu vào (input)
- Hàm này nhận 3 tham số:
    1. [args]: Thông tin đơn hàng:
        - URL là gì?
        - Method là gì? GET hay POST...
        - Body có gì?
    2. [api]: Các công cụ hỗ trợ của Redux (như [dispatch], [getState] để lấy token từ store nếu cần)
    3. [extraOptions]: Các tuỳ chọn phụ (ít dùng)

## B. Đầu ra (Output)
- Ông QUẢN LÝ KHO RTK Query rất khó tính, ông ấy chỉ hiểu đúng 2 kiểu trả về:

In [ ]:
/* Nếu thành công --> Phải trả về object có key là `data` */

return { data: { id: 1, name: "Sản phẩm A" } };

/* Nếu thất bại --> Phải trả về object có key là `error` */
return { error: { status: 404, data: "Không tìm thấy" } };

// Nếu trả về sai quy tắc này (ví dụ trả về trực tiếp response của Axios), RTK Query sẽ không hiểu và lỗi

In [ ]:
// VÍ DỤ CHUẨN cầu nối để biến Axios thành baseQuery chuẩn của RTK Query

// Import cái Axios Instance đã cấu hình sẵn (có interceptor)
import httpRequest from "@/utils/httpRequest";

// Đây là hàm Custom Base Query
const baseQuery = async (args, api, extraOptions) => {
    try {
        // 1. Lấy thông tin request từ args
        // args có thể là chuỗi "/products" hoặc object { url:..., method:... }
        const { url, method, body, params } = args;

        // 2. Gọi Shipper Axios đi làm việc
        const result = await httpRequest({
            url: url,
            method: method || "GET",
            data: body,
            params: params,
        });

        // 3. Quy tắc Output: Trả về dạng { data: ... }
        // Vì httpRequest của bạn đã trả về data (do interceptor), nên gán nó vào key data
        return { data: result }; 

    } catch (axiosError) {
        // 4. Quy tắc Output: Nếu lỗi, trả về dạng { error: ... }
        return {
            error: {
                status: axiosError.response?.status,
                data: axiosError.response?.data || axiosError.message,
            },
        };
    }
};

export default baseQuery;

In [ ]:
# TÓM LẠI
    1. Tại sao cần đọc docs này?
        --> Để biết cách thay thế [fetch] mặc định bằng [axios]
        
    2. Mấu chốt: Viết một hàm trung gian (baseQuery)
        - Nó nhận lệnh từ RTK baseQuery
        - Nó sai Axios đi lấy dữ liệu
        - Nó nhận kết quả từ Axios, đóng gói lại thành {data: ....} hoặc {error: ...} rồi đưa cho RTK Query